In [8]:
import itk
from itk import TubeTK as ttk

from itkwidgets import view

import numpy as np

In [9]:
ImageType = itk.Image[itk.F, 3]
dir = "../data/CTA/"
im1iso = itk.imread(dir + "cta.mha")
im1BrainVess = itk.imread(dir + "cta_brain_vessels.mha")

In [10]:
imMath = ttk.ImageMath.New(im1BrainVess)
imMath.MedianFilter(1)
imMath.Threshold(0.00000001, 1, 1, 0)
im1VessMask = imMath.GetOutputShort()

ccSeg = ttk.SegmentConnectedComponents.New(im1VessMask)
ccSeg.SetMinimumVolume(50)
ccSeg.Update()
im1VessMaskCC = ccSeg.GetOutput()

In [11]:
view(im1VessMaskCC)

Viewer(geometries=[], gradient_opacity=0.22, point_sets=[], rendered_image=<itk.itkImagePython.itkImageSS3; pr…

In [12]:
imMathSS = ttk.ImageMath.New(im1VessMaskCC)
imMathSS.Threshold(0,0,1,0)
im1VessMaskInv = imMathSS.GetOutputFloat()

distFilter = itk.DanielssonDistanceMapImageFilter.New(im1VessMaskInv)
distFilter.Update()
dist = distFilter.GetOutput()

imMath.SetInput(dist)
imMath.Blur(0.4)
tmp = imMath.GetOutput()
imMath.ReplaceValuesOutsideMaskRange(tmp, 0.1, 10, 0)
im1SeedRadius = imMath.GetOutput()

itk.imwrite(im1SeedRadius, dir+"cta_vessel_seed_radius.mha")

In [13]:
view(im1SeedRadius)

Viewer(geometries=[], gradient_opacity=0.22, point_sets=[], rendered_image=<itk.itkImagePython.itkImageF3; pro…

In [14]:
imMath.SetInput(im1iso)
imMath.ReplaceValuesOutsideMaskRange(im1BrainVess, 0, 1000, 0)
imMath.Blur(0.4)
imMath.IntensityWindow(0.5,300,0,300)
im1Input = imMath.GetOutput()

itk.imwrite(im1iso, dir+"cta_vessels_segmentation_input.mha")

view(im1Input)

Viewer(geometries=[], gradient_opacity=0.22, point_sets=[], rendered_image=<itk.itkImagePython.itkImageF3; pro…

In [17]:
numSeeds = 40

vSeg = ttk.SegmentTubes.New(Input=im1Input)
vSeg.SetVerbose(True)
vSeg.SetMinCurvature(0)#.0001)
vSeg.SetMinRoundness(0.02)
vSeg.SetMinRidgeness(0.5)
vSeg.SetMinLevelness(0.0)
vSeg.SetRadiusInObjectSpace( 0.8 )
vSeg.SetBorderInIndexSpace(3)
vSeg.SetSeedMask( im1SeedRadius )
#vSeg.SetSeedRadiusMask( im1SeedRadius )
vSeg.SetOptimizeRadius(True)
vSeg.SetUseSeedMaskAsProbabilities(True)
vSeg.SetSeedExtractionMinimumProbability(0.4)
#vSeg.SetSeedMaskMaximumNumberOfPoints( numSeeds )
vSeg.ProcessSeeds()

In [18]:
tubeMaskImage = vSeg.GetTubeMaskImage()
view(tubeMaskImage)

Viewer(geometries=[], gradient_opacity=0.22, point_sets=[], rendered_image=<itk.itkImagePython.itkImageF3; pro…

In [19]:
SOWriter = itk.SpatialObjectWriter[3].New()
SOWriter.SetInput(vSeg.GetTubeGroup())
#SOWriter.SetBinaryPoints(True)
SOWriter.SetFileName( dir+"cta_vessels.tre" )
SOWriter.Update()

In [20]:
VTPWriter = itk.WriteTubesAsPolyData.New()
VTPWriter.SetInput(vSeg.GetTubeGroup())
VTPWriter.SetFileName(dir+"cta_vessels.vtp")
VTPWriter.Update()